## Load Data

In [49]:
# !pip install nltk

     |████████████████████████████████| 1.5 MB 7.8 MB/s eta 0:00:01
     |████████████████████████████████| 75 kB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 284 kB 40.8 MB/s eta 0:00:01


In [57]:
import numpy as np
import pandas as pd
import warnings
import nltk
warnings.filterwarnings('ignore')
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/garismendi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [69]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

print(df_train.describe())
print(df_train.head())

                 id      target
count   7613.000000  7613.00000
mean    5441.934848     0.42966
std     3137.116090     0.49506
min        1.000000     0.00000
25%     2734.000000     0.00000
50%     5408.000000     0.00000
75%     8146.000000     1.00000
max    10873.000000     1.00000
   id keyword location                                               text  \
0   1     NaN      NaN  Our Deeds are the Reason of this #earthquake M...   
1   4     NaN      NaN             Forest fire near La Ronge Sask. Canada   
2   5     NaN      NaN  All residents asked to 'shelter in place' are ...   
3   6     NaN      NaN  13,000 people receive #wildfires evacuation or...   
4   7     NaN      NaN  Just got sent this photo from Ruby #Alaska as ...   

   target  
0       1  
1       1  
2       1  
3       1  
4       1  


In [61]:
df_train[df_train['target'] == 0]['text'].values[0]

"What's up man?"

In [62]:
df_train[df_train['target'] == 1]['text'].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [63]:
df_train['text'].values[0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [64]:
print(df_train['target'].value_counts())

0    4342
1    3271
Name: target, dtype: int64


## Data Preprocessing

In [65]:
porter = PorterStemmer()
sw = stopwords.words('english')

def porter_stemmer(text):
    return [porter.stem(word) for word in text.split() if word not in sw]

In [70]:
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer(stop_words='english', lowercase=False)
train_vectors = tfidf_vectorizer.fit_transform(df_train['text'])

print(train_vectors[0].todense().shape)
print(train_vectors[0].todense())

(1, 25532)
[[0. 0. 0. ... 0. 0. 0.]]


In [71]:
test_vectors = tfidf_vectorizer.transform(df_test['text'])
print(test_vectors[0].todense().shape)
print(test_vectors[0].todense())

(1, 25532)
[[0. 0. 0. ... 0. 0. 0.]]


## Train Models

In [43]:
import xgboost
print(xgboost.__version__)
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

1.3.3


In [44]:
# Create models
xgb_clf = XGBClassifier(booster='gbtree', objective="binary:logistic", use_label_encoder=False, cv=3, random_state=42)
lr = LogisticRegression(random_state=42)
svc = SVC(random_state=42) 
models = [('XGBoost',xgb_clf), ('Logistic Regression', lr), ('SVC', svc)]

In [45]:
for name, algo in models:
    scores = model_selection.cross_val_score(algo, train_vectors, df_train['target'], cv=5, scoring='f1')
    print(f'{name}: {scores}')

[09:08:47] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { cv } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[09:08:47] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[09:08:49] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { cv } might not be used.

  This may not b

In [12]:
# clf.fit(train_vectors, df_train["target"])

[23:48:56] WARNING: /tmp/build/80754af9/xgboost-split_1619724447847/work/src/learner.cc:541: 
Parameters: { cv } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, cv=3, gamma=0, gpu_id=-1,
       importance_type='gain', interaction_constraints='',
       learning_rate=0.300000012, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=nan, monotone_constraints='()',
       n_estimators=100, n_jobs=8, num_parallel_tree=1,
       objective='reg:squarederror', random_state=42, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
       use_label_encoder=False, validate_parameters=1, verbosity=None)

In [13]:
# sample_submission = pd.read_csv('sample_submission.csv')

In [14]:
# sample_submission['target'] = clf.predict(test_vectors)
# sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [15]:
# sample_submission.to_csv('submission.csv', index=False)